Now we calculate the q-value $Q$ where 
\begin{equation}
\begin{split}
Q = \Delta mc^2\\
\end{split}
\end{equation}
where $c$ is the speed of light, and $\Delta m$ is the change in mass of the all the nuclei in a reaction. 
For these reactions, the only reactants are the target and the projectile. 






In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using PlotlyJS, CSV, DataFrames
using Printf
using SymPy
using PDFIO
using Unzip
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

WebIO._IJuliaInit()

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

In [2]:
function is_number(input) 
    if isa(input, Number)
        return true
    end
    return tryparse(Float64, input) != nothing
end

is_number (generic function with 1 method)

Now to import and clean the data

In [3]:
amu_path = parent_dir * "ImportedData\\Atomic_mass_table_2020.csv"
masses = CSV.read(amu_path, DataFrame, stringtype=String)[:,2:end]
good_rows = [row for row in 1:size(masses)[1] if is_number(masses[row, end-1])]
masses = masses[good_rows, :]
masses[!,1:3] = Int.(masses[:,1:3])
masses[:,[3,4,end-1]]

,A,Elt.,Atomic mass (u)
,Int64,String,Float64
1,1,n,1.00866
2,1,H,1.00783
3,2,H,2.0141
4,3,H,3.01605
5,3,He,3.01603
6,3,Li,3.03078
7,4,H,4.02643
8,4,He,4.0026
9,4,Li,4.02719


This function returns the precise mass in daltons of a given isotope. Isotopes must be given in the form mass number + symbol in a single string, such as "3H" for tritium or "1n" for a free neutron. 

In [4]:
function precise_mass(input)
    index = 1
    while (index < length(input) && 
            tryparse(Int, string(input[1:index])) != nothing)
        index += 1
    end
    index -= 1
    mass_number = parse(Int, input[1:index])
    symbol = input[index+1:end]
    row = 1
    while ((masses[row,3] != mass_number || masses[row,4] != symbol) 
        && row < size(masses)[1])
        row += 1
    end
    if row == size(masses)[1]
        if string(masses[row,3]) * string(masses[row,4]) == input
            return masses[row, end-1]
        else 
            return "Isotope not in dataset"
        end
    end
    return masses[row, end-1] 
end
precise_mass("3H")

3.01604928132

In [5]:
N_A = 6.02214076*(BigFloat(10)^23)

6.02214076000000009258883437723852694034576416015625e+23

The function q_val returns the q value of a reaction given the products and reactants in eV/reaction. 

In [6]:
kg_per_dalton = 1.66053906660 * BigFloat(10)^(-27)
c = 299792458
joule_per_eV = 1.602176634 * BigFloat(10)^-19
conversion_constant = kg_per_dalton * c^2 / joule_per_eV
total_mass(ingredients) = sum([precise_mass(ingredient) for ingredient in ingredients])
q_val(reactants, products) = (total_mass(reactants) - total_mass(products)) * conversion_constant 
q_val(["3H", "2H"], ["4He","1n"])

1.758929986491761040539044986485839498059167757678315342281902419445550953510919e+07

This agrees with the usually cited q-values for different reactions. 

In [7]:
q_val(["2H", "2H"], ["3He", "1n"])

3.268908850974102367854075994107107301986568434178828839895100956710973926549135e+06

In [8]:
q_val(["2H", "2H"], ["3H", "1H"])

4.032663828738721139338876004089564883296793627391702351816031599817901716929761e+06

In [9]:
q_val(["1n", "9Be"],["8Be", "1n", "1n"])

-1.664538882129621392511337106953757283363676677527588068640109772397040296181692e+06

In [10]:
df = CSV.read(parent_dir * "ExportedData\\mt_reactions.csv", 
                        DataFrame, stringtype=String)
mt_reaction_df = DataFrame()
mt_reaction_df[!, "MT"] = df[!,1]
mt_reaction_df[!, "Reaction"] = df[!,2]
mt_reaction_df

,MT,Reaction
,Int64,String
1,875,"(z,2n0)"
2,35,"(z,nd2α)"
3,114,"(z,d2α)"
4,30,"(z,2n2α)"
5,700,"(z,t0)"
6,699,"(z,dc)"
7,32,"(z,nd)"
8,650,"(z,d0)"
9,117,"(z,dα)"


I think I need to programmatically redo the interpretation of MTs to account for multiple sets of nuclei and free particles having the same reaction. 

function MTs(projectile, reaction)
    df = projectile_MT_dict[projectile]
    row = 1
    while (row < size(df)[1] && reaction != df[row, "Reaction"])
        row += 1
    end
    if row == size(df)[1]
        if !(reaction == df[row, "Reaction"])
            return "Reaction not in dataset"
        end
    end
    return df[row, "MTs"]
end
MTs("Proton", [0,0])

In [22]:
element_symbols = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", 
"Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", 
"Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", 
"Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", 
"Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", 
"Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", 
"Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", 
"Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", 
"Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", 
"Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"];
function z_a_format(isotope)
    char_array = collect(isotope)
    index = 1
    try
        while isdigit(char_array[index])
            index += 1
        end
        index -= 1
        a = parse(Int64, join(char_array[1:index]))
        z_as_string = join(char_array[index+1:end])
        if z_as_string == "n"
            z = 0
        else
            z = findfirst(x->x == z_as_string, element_symbols)
        end
        return (z, a)
    catch 
        error_message = "Error in isotope " * isotope
        println(error_message)
        return error_message
    end 
end

function isotope_name(z_a)
    try
        z, a = z_a
        symbol = element_symbols[z]
        return string(a) * symbol
    catch
        return ("Error with isotope " * string(z_a) * " expected (atomic number, mass number)")
    end
end

isotope_name (generic function with 1 method)

In [23]:
mt_df = CSV.read(parent_dir * "ExportedData\\mt_reactions.csv", DataFrame)
mt_reactions = Dict([])
for row in 1:size(mt_df)[1]
    mt_reactions[mt_df[row,1]] = mt_df[row, 2]
end
mt_reactions

Dict{Any, Any} with 103 entries:
  875 => "(z,2n0)"
  35  => "(z,nd2α)"
  114 => "(z,d2α)"
  30  => "(z,2n2α)"
  700 => "(z,t0)"
  699 => "(z,dc)"
  32  => "(z,nd)"
  650 => "(z,d0)"
  117 => "(z,dα)"
  45  => "(z,npα)"
  215 => "(z,Xκ0 short)"
  251 => "(n,...)"
  253 => "(n,...)"
  115 => "(z,pd)"
  112 => "(z,pα)"
  210 => "(z,Xπ−)"
  151 => "(n,RES)"
  90  => "(z,n40)"
  457 => "(z,...)"
  460 => "(z,...)"
  4   => "(z,n)"
  207 => "(z,Xα)"
  104 => "(z,d)"
  91  => "(z,nc)"
  205 => "(z,Xt)"
  ⋮   => ⋮

TODO: Make the code below more concise. It looks likes spaghetti. 

I need to modify the above code to also return the specific nuclei and free particles involved in a reaction. 

In [15]:
MTs = [key for key in keys(mt_reactions)]
reactions = [value for value in values(mt_reactions)]
symbol_dict = Dict([
            ("n", (0, 1)),
            ("t", (1, 3)), #2 neutrons, 1 proton
            ("d", (1, 2)),
            ("p", (1, 1)),
            ("α", (2, 4)),
            ("γ", (0, 0)),
            ("3 He", (2, 3)),
            ('c', (0,0)) #reverse to continuum emission
        ])
symbol_to_nucleus = Dict([
    ("n", "1n"),
    ("t", "3H"), #2 neutrons, 1 proton
    ("d", "2H"),
    ("p", "1H"),
    ("α", "4He"),
    ("γ", nothing),
    ("3 He", "3He"),
    ("c", nothing) #reverse to continuum emission
])
reaction_symbols = [key for key in keys(symbol_dict)]

function find_difference(term, projectile_z_a = (0,0), projectile_symbol = "c")
    #e.g. term = "n"
    difference = (0, 0) #(Z, A)
    nuclei = []
    if occursin("z", term)
        difference = (projectile_z_a[1], projectile_z_a[2])
        push!(nuclei, symbol_to_nucleus[projectile_symbol])
    end
    #now to search for instance of any of the keys in term
    for key in reaction_symbols
        if occursin(key, term)
            sub_difference = (0, 0)
            key_index = collect(findfirst(key, term))[1]
            push!(nuclei, symbol_to_nucleus[key])
            sub_difference = symbol_dict[key]
            #check for coefficient
            if key_index > 1 && isdigit(term[key_index-1])
                coefficient = parse(Int64, term[key_index-1])
                for i in 1:coefficient-1
                    push!(nuclei, symbol_to_nucleus[key])
                end
                sub_difference = (sub_difference[1] * coefficient, 
                    sub_difference[2] * coefficient)
            end
            difference = (difference[1] + sub_difference[1], difference[2] + sub_difference[2])
        end
    end
    return difference, nuclei     
end
  
projectile_z_a_dict = Dict(["electron" => (0,0), "neutron"  => (0,1), 
    "proton" => (1,1), "deuteron" => (1,2), "tritium" => (1,3), 
    "alpha" => (2,2), "gamma" => (0,0), "3He"  => (2,3)])
projectile_symbol_dict = Dict(["proton" => "p", "neutron" => "n",
    "alpha" => "α", "gamma" => "γ", "3He" => "3 He", "deuteron" => "d",
    "tritium" => "t"])

function interpret_reaction(reaction, projectile)
    projectile_z_a = projectile_z_a_dict[projectile]
    projectile_symbol = projectile_symbol_dict[projectile]
    comma_index = collect(findfirst(",", reaction))[1]
    reaction = collect(reaction)
    added = reaction[2:comma_index-1]
    ejected = reaction[comma_index+1:length(reaction)-1]
    #the valid option are n, t, d, p, α, γ, He
    # a number followed by a letter is a coefficient
    # a letter followed by a number is an energy level.
    # a letter followed by c means continuum emission of that particle
    #each value will be added or subtracted from the target
    added_difference, added_nuclei = find_difference(String(added),
                                        projectile_z_a, projectile_symbol)
    ejected_difference, ejected_nuclei = find_difference(String(ejected), 
                                            projectile_z_a, projectile_symbol)
    deltaZ_deltaA = (added_difference[1] - ejected_difference[1], 
                added_difference[2] - ejected_difference[2])
    return deltaZ_deltaA, added_nuclei, ejected_nuclei
end
projectile = "proton"
react = mt_reactions[875]
interpret_reaction(react, "proton")

((1, -1), Any["1H"], Any["1n", "1n"])

In [16]:
react

"(z,2n0)"

This function find_ingredients returns the reactants and products given the projectile, MT (as defined in the ENDF6 manual; a copy of the relevant section is in the ImportedData directory and named MT.pdf), and target. 

In [38]:
function find_ingredients(projectile, MT, target) 
    reaction = mt_reactions[MT] 
    deltaZ_deltaA, reactants, products = interpret_reaction( 
            reaction,  projectile) 
    push!(reactants, target)
    target = z_a_format(target)
    target_remnant = (target[1] + deltaZ_deltaA[1], target[2] + deltaZ_deltaA[2])
    push!(products, isotope_name(target_remnant))
    return reactants, products
end
find_ingredients("proton", 875, "9Be")

(Any["1H", "9Be"], Any["1n", "1n", "8B"])

In [34]:
mt_reactions[4]

"(z,n)"

In [39]:
q_val(reactants_products_list) = q_val(reactants_products_list[1], reactants_products_list[2])
q_val(projectile, MT, target) = q_val(find_ingredients(projectile, MT, target))
q_val("deuteron", 4, "3H")

1.758929986491761040539044986485839498059167757678315342281902419445550953510919e+07

It works! :)

In [64]:
function find_corresponding_MTs(projectile_name)  
    projectile_symbol = projectile_symbol_dict[projectile_name]  
    projectile_z_a = projectile_dict[projectile_name]
    interpreted_MTs = Dict([])
    for MT in MTs
        try
            interpreted_MTs[MT], added_nuclei, ejected_nuclei = interpret_reaction(
                mt_reactions[MT], projectile_z_a, projectile_symbol)
        catch
            println(MT)
        end
    end
    #details = [value for value in values(mt_details)]
    #will write test cases later
    interpretations = [value for value in values(interpreted_MTs)]
    #there are many reactions that yield the same nucleus. 
    possible_changes_in_nucleus = unique(interpretations)
    corresponding_MTs = Dict([])
    for change in possible_changes_in_nucleus
        corresponding_MTs[change] = [key for key in keys(interpreted_MTs) 
                                if interpreted_MTs[key] == change]
    end
    return corresponding_MTs
end

gamma_MTs = find_corresponding_MTs("gamma")
proton_MTs = find_corresponding_MTs("proton")
alpha_MTs = find_corresponding_MTs("alpha")
tritium_MTs = find_corresponding_MTs("tritium")
neutron_MTs = find_corresponding_MTs("neutron")
deuteron_MTs = find_corresponding_MTs("deuteron")
helium3_MTs = find_corresponding_MTs("3He")

699
91
849
10
799
749
891
649
699
91
849
10
799
749
891
649
699
91
849
10
799
749
891
649
699
91
849
10
799
749
891
649
699
91
849
10
799
749
891
649
699
91
849
10
799
749
891
649
699
91
849
10
799
749
891
649


Dict{Any, Any} with 26 entries:
  (-4, -9)  => [109]
  (1, 2)    => [603, 203, 103, 217, 600, 601, 604, 602]
  (0, -3)   => [24]
  (2, -1)   => [37]
  (-1, -3)  => [117, 45]
  (-2, -6)  => [29]
  (-3, -9)  => [36]
  (0, -2)   => [38, 22]
  (2, 0)    => [17]
  (-1, -2)  => [112]
  (2, 1)    => [875, 16, 876]
  (-6, -12) => [23]
  (-3, -7)  => [114]
  (1, -1)   => [11, 42, 33]
  (0, -1)   => [207, 801, 50, 116, 107, 21, 800]
  (2, 2)    => [90, 4, 218, 52, 34, 214, 51]
  (-3, -8)  => [35, 113]
  (1, 0)    => [700, 32, 215, 205, 41, 702, 105, 701]
  (2, 3)    => [210, 457, 460, 456, 451, 216, 208, 211, 202, 455, 213, 450, 106…
  (0, 0)    => [115, 2, 20, 44]
  (0, -4)   => [25]
  (-2, -7)  => [30]
  (-2, -5)  => [108]
  (1, 1)    => [650, 104, 651, 28, 204, 652]
  (0, 1)    => [251, 253, 151, 111, 751, 750, 27, 458, 19, 252]
  (2, -2)   => [152]

In [23]:
projectile = "proton"
interpret_reaction(mt_reactions[875], projectile_dict[projectile])

((1, -1), Any[], Any["1H", "1H"])

In [ ]:
interpret_reaction(mt_reactions[849], (1,1))

In [ ]:
get_ingredients(projectile, MT)
    